In [ ]:
# value function iteration
function vfi(vmat0,bgrid,Ge,Pe,β,q)

    Ne = size(vmat0,1)
    Nb = size(vmat0,2)

    # 0. construct the current utility
    umat = zeros(Ne,Nb,Nb)

    [...]
    
    gmat0 = zeros(ne,nb)     # policy function
    vmat1 = zeros(ne,nb)     # new value function

    diff = 1e+4
    it = 0 # counter

    while diff>1e-7 # while loop until the condition diff>1e-4 met

        for k in 1:Ne # for each k=0,1
            # 1-1. compute the expected value function in the next period
            [...]
            
            for i in 1:Nb # for each i=0,...,nb-1
                # 1-2. find the index j so as to maximize the value function
                # NOTE: gmat0 stores indices for bgrid
                [...]
            end
            
        end

        diff = maximum(abs.(vmat1-vmat0)) # the condition for convergence
        it   = it + 1
        # 2. update the value function       
        vmat0 = copy(vmat1)
        
    end
        
    return vmat0, gmat0
    
end

In [3]:
# transition matrix
function transition_matrix(gmat0,Pe)

    Ne = size(gmat0,1)
    Nb = size(gmat0,2)
    A  = zeros(Ne*Nb,Ne*Nb)

    for k in 1:Ne         # index for today's employment

        for i in 1:Nb     # index for tomorrow's asset

            j = Int(gmat0[k,i]) # index for tomorrow's asset, from the policy function

            for l in 1:Ne # index for tomorrow's employment

                m = Nb*(k-1)+i # m = 1,...,2Nb
                n = Nb*(l-1)+j # n = 1,...,2Nb
                A[m,n] = Pe[k,l]
                
            end
            
        end
        
    end
                
    return A
    
end

In [4]:
# distribution
function dist_iter(lambda0,A)

    N = size(lambda0,1)
    lambda1 = zeros(N)

    diff = 1e+4
    
    while diff>1e-14
        lambda1 = A'*lambda0
        diff    = maximum(abs.(lambda1-lambda0))
#         print(diff)
        lambda0 = lambda1/sum(lambda1)
    end

    return lambda0
    
end

In [5]:
# setup
# parameter values
β   = 0.96
μ   = 0.1 # unemployment insurance
pee = 0.925
puu = 0.5
ϕ   = 2   # borrowing constraint

# set grid
nb = 1001
bmin = -ϕ
bmax = 12.0
bgrid = collect(range(bmin,stop=bmax,length=nb))

ne = 2
Ge = [1.0,μ]
Pe = [pee 1-pee;
    1-puu puu]

In [6]:
q = 0.96

vmat0 = np.zeros((ne,nb))                   # initial (old) value function
lambda0 = np.ones(ne*nb)/(ne*nb)            # initial distribution

start = time.time()
vmat0, gmat0 = vfi(vmat0,bgrid,Ge,Pe,β,q)   # value function iteration
A            = transition_matrix(gmat0,Pe)  # transition matrix
lambda0      = dist_iter(lambda0,A)         # stationary distribution
# the aggregate demand for asset
# NOTE: lambda is (2*nb times 1) vector, the first nb elements are for k=0 and the second nb elements are for k=1
z = np.sum(bgrid*lambda0[0:nb]) + np.sum(bgrid*lambda0[nb:2*nb])
print(np.array([q,z]))
eptime = time.time()-start

[0.96       7.90325303]
